In [3]:
#hide

from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#from pathlib import Path

datap = Path('./kaggle/ssts-forecasting')


for dirname, _, filenames in os.walk(datap):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train_data = pd.read_csv(datap / "train.csv")
train_data.head()
train_data.columns

FileNotFoundError: [Errno 2] No such file or directory: 'kaggle/ssts-forecasting/train.csv'

In [19]:
test_data = pd.read_csv(datap / "test.csv")
test_data.head()
test_data.columns

Index(['id', 'date', 'store_nbr', 'family', 'onpromotion'], dtype='object')

# fastai method

In [20]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [29]:
procs = [Categorify]
features = [
    'id', 'date', 'store_nbr', 'family', 'onpromotion']
dep_var = 'sales'
# all_data = pd.concat([train_data, test_data], sort=False)
# It looks to me like the test data has no survival column
all_data = train_data[features + [dep_var]]

cont, cat = cont_cat_split(all_data, 1, dep_var=dep_var)

split_idx = int(math.floor(len(all_data) * 0.70))
splits = (list(range(0, split_idx)), 
          list(range(split_idx, len(all_data))))

In [30]:
to = TabularPandas(
    all_data[features + [dep_var]],
    procs,
    cat,
    cont,
    y_names=dep_var,
    splits=splits
)
to.show(10)

# save_pickle(datap / 'to.pkl', to)
# to = (datap / 'to.pkl').load()

xs,y = to.train.xs, to.train.y
valid_xs,valid_y = to.valid.xs, to.valid.y

,date,family,id,store_nbr,onpromotion,sales
0,2013-01-01,AUTOMOTIVE,0,1,0,0.0
1,2013-01-01,BABY CARE,1,1,0,0.0
2,2013-01-01,BEAUTY,2,1,0,0.0
3,2013-01-01,BEVERAGES,3,1,0,0.0
4,2013-01-01,BOOKS,4,1,0,0.0
5,2013-01-01,BREAD/BAKERY,5,1,0,0.0
6,2013-01-01,CELEBRATION,6,1,0,0.0
7,2013-01-01,CLEANING,7,1,0,0.0
8,2013-01-01,DAIRY,8,1,0,0.0
9,2013-01-01,DELI,9,1,0,0.0


In [39]:
m = DecisionTreeRegressor(min_samples_leaf=10)
m.fit(xs, y)

m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

(269.138214, 547.068046)

In [40]:
test_to = TabularPandas(
    test_data[features],
    procs,
    cat,
    cont
)
test_to.show(3)
test_cleaned = test_to.train.xs

,date,family,id,store_nbr,onpromotion
0,2017-08-16,AUTOMOTIVE,3000888,1,0
1,2017-08-16,BABY CARE,3000889,1,0
2,2017-08-16,BEAUTY,3000890,1,2


In [41]:
predictions = m.predict(test_cleaned);
predictions[0:10]

array([   5.        ,    0.        ,    7.36842105, 2369.14285714,
          0.        ,  380.13066228,   14.        ,  854.07142857,
        854.07142857,  130.45626349])

In [43]:
output = pd.DataFrame(
    {
        'id': test_cleaned.id,
        'sales': predictions,
    }
)
output

,id,sales
0,3000888,5.000000
1,3000889,0.000000
2,3000890,7.368421
3,3000891,2369.142857
4,3000892,0.000000
...,...,...
28507,3029395,408.866999
28508,3029396,82.983400
28509,3029397,1279.750586
28510,3029398,90.398313


In [44]:
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
